In [ ]:
import random
import math
from statistics import mean

def is_max_die_value(dice_str, roll_value):
    num_dice, num_faces = map(int, dice_str.split('d'))
    return roll_value == num_faces    



In [ ]:
class DiceRoller:
    def roll(dice_str, advantage=0, print_results=False):
        # ABSTRACT INTERFACE FOR ROLLING DICE
        # SHOULD RETURN A LIST OF INTS REPRESENTING THE DICE RESULTS
        pass
        
    def roll_as_modifier(dice_str, advantage=0, print_results=False):
        # ABSTRACT INTERFACE FOR ROLLING DICE
        # SHOULD RETURN A LIST OF INTS REPRESENTING THE DICE RESULTS
        pass

In [ ]:
class NimbleDiceRoller(DiceRoller):
    def roll(self, dice_str, advantage=0, print_results=False):
        try:
            # Split the input string into number of dice and number of faces
            num_dice, num_faces = map(int, dice_str.split('d'))

            if print_results:
                print(f'Rolling {dice_str}...')
            
            num_advantage_stacks = abs(advantage)
            num_dice += num_advantage_stacks

            results = [random.randint(1, num_faces) for _ in range(num_dice)]

            if print_results:
                print(f'Original roll: {results}')
                
            for _ in range(num_advantage_stacks):
                if advantage > 0:
                    results.remove(min(results))
                else:
                    results.remove(max(results))

            if print_results:
                print(f'After advantage: {results}')

            is_crit = results[0] == num_faces
            while is_crit:
                results.append(random.randint(1, num_faces))
                is_crit = results[-1] == num_faces
    
            if print_results:
                print(f'After exploding: {results}')

            return results
        
        except ValueError:
            # Handle the case where the input values are not valid
            return "Invalid input values. Please use integers for num_dice and num_faces."

    def roll_as_modifier(self, dice_str, advantage=0, print_results=False):
        try:
            # Split the input string into number of dice and number of faces
            num_dice, num_faces = map(int, dice_str.split('d'))

            if print_results:
                print(f'Rolling {dice_str}...')
            
            num_advantage_stacks = abs(advantage)
            num_dice += num_advantage_stacks

            results = [random.randint(1, num_faces) for _ in range(num_dice)]

            if print_results:
                print(f'Original roll: {results}')
                
            for _ in range(num_advantage_stacks):
                if advantage > 0:
                    results.remove(min(results))
                else:
                    results.remove(max(results))

            if print_results:
                print(f'After advantage: {results}')

            return results
        
        except ValueError:
            # Handle the case where the input values are not valid
            return "Invalid input values. Please use integers for num_dice and num_faces."

NimbleDiceRoller().roll("3d4", advantage=0, print_results=True)

In [ ]:
def attack_roll(dice_str, advantage, dice_roller):
    results = dice_roller.roll(dice_str, advantage)
    if results[0] == 1:
        return 0
    return sum(results)

def get_average_damage(dice_str, advantage, dice_roller, num_tests=100000):
    average = 0
    for _ in range(num_tests):
        damage = attack_roll(dice_str, advantage, dice_roller)
        average += damage / num_tests
    return average

# print(get_average_damage("1d4", 0, NimbleDiceRoller(), 100000))

In [ ]:
class Character:
    def __init__(self, name, hp, ac, num_attacks, attack_dice, damage_modifier):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.num_attacks = num_attacks
        self.attack_dice = attack_dice
        self.damage_modifier = damage_modifier

In [ ]:
fighter = Character(name="Fighter 3", hp=40, ac=18, num_attacks=1, attack_dice="1d8", damage_modifier=3)
monster = Character(name="Light Armor Monster 3", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)

In [ ]:
class AttackProcessor:
    def start_of_round(self):
        # PROCESSING AT START OF ROUND
        pass
        
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results = False):
        # ABSTRACT DEFINITION FOR PERFORMING ATTACKS
        pass

class DefenseProcessor:
    def start_of_round(self):
        # PROCESSING AT START OF ROUND
        pass
        
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        # ABSTRACT DEFINITION FOR CHECKING IF AN ATTACK HIT
        pass
        
    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        # ABSTRACT DEFINITION FOR DEFENDING ATTACKS
        pass

In [ ]:
def get_damage_for_round(attacker, defender, attack_processor, defense_processor, dice_roller, print_results=False):
    attack_processor.start_of_round()
    defense_processor.start_of_round()
    damage = attack_processor.perform_attacks(attacker, defender, defense_processor, dice_roller, print_results)
    return damage

In [ ]:
class NimbleStandardAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [ ]:
class NimbleRogueAttack(AttackProcessor):
    def __init__(self, offhand_dice_str, sneak_attack_dice_str):
        self.offhand_dice_str = offhand_dice_str
        self.sneak_attack_dice_str = sneak_attack_dice_str
        
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process offhand attack
        damage = self.perform_single_attack(attacker, defender, defense_processor, self.offhand_dice_str, 0, dice_roller, 0, print_results)
        total_damage += damage        
        
        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage

        if total_damage > 0:
            damage = sum(dice_roller.roll_as_modifier(self.sneak_attack_dice_str, 0, print_results))
            if print_results:
                print(f'Sneak Attack for {damage} damage')
                
            total_damage += damage            
        
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [ ]:
class NimbleMonsterLightArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        return damage
        
print("FIGHTER")
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))

print("\nROGUE")
print(get_damage_for_round(fighter, monster, NimbleRogueAttack(offhand_dice_str="1d4", sneak_attack_dice_str="2d6"), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))


In [ ]:
class NimbleMonsterMediumArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls)

        if was_crit:
            damage += damage_modifier
        
        return damage

print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterMediumArmorDefense(), NimbleDiceRoller(), print_results=True))

In [ ]:
class NimbleMonsterHeavyArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        
        if not was_crit:
            damage = math.ceil(damage / 2)
        
        return damage
        
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterHeavyArmorDefense(), NimbleDiceRoller(), print_results=True))

In [ ]:
class NimbleMonsterArmorDefense(DefenseProcessor):
    def __init__(self, monster):
        if monster.ac < 14:
            self.armor_processor = NimbleMonsterLightArmorDefense()
        elif monster.ac < 18:
            self.armor_processor = NimbleMonsterMediumArmorDefense()
        else:
            self.armor_processor = NimbleMonsterHeavyArmorDefense()

    def start_of_round(self):
        self.armor_processor.start_of_round()

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        return self.armor_processor.did_attack_hit(attacker, defender, attack_dice, print_results)

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        return self.armor_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)

monster.ac = 12
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 15
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 19
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))


monster.ac = 12
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 15
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 19
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))

In [ ]:
class NimbleMonsterAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, dice_roller, 0, print_results)
            total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attacker.attack_dice, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attacker.attack_dice}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attacker.attack_dice, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, attacker.damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [ ]:
class NimblePlayerDefense(DefenseProcessor):
    def __init__(self):
        self.has_blocked = False

    def start_of_round(self):
        self.has_blocked = False

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        
        if not self.has_blocked:
            damage_reduction = defender.ac - 8

            if print_results:
                print(f'{defender.name} blocks {damage_reduction} damage')
            
            damage = max(0, damage - damage_reduction)
            self.has_blocked = True
        
        return damage

print(get_damage_for_round(monster, fighter, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller(), print_results=True))

In [ ]:
def get_dpr(attacker, defender, attack_processor, defense_processor, dice_roller, num_simulations=100000):
    results = [get_damage_for_round(attacker, defender, attack_processor, defense_processor, dice_roller) for _ in range(num_simulations)]
    return mean(results)
    average_damage = 0.0
    for _ in range(num_simulations):
        damage = get_damage_for_round(attacker, defender, attack_processor, defense_processor, dice_roller)
        average_damage += damage / num_simulations
    return average_damage

In [ ]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Fighter 3", hp=40, ac=18, num_attacks=1, attack_dice="1d8", damage_modifier=3)
monster = Character(name="Monster", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr = get_dpr(player, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("FIGHTER")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))


In [ ]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Rogue 3", hp=40, ac=16, num_attacks=1, attack_dice="1d6", damage_modifier=3)
monster = Character(name="Monster", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr = get_dpr(player, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("ROGUE")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))


In [ ]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Wizard 3", hp=40, ac=12, num_attacks=1, attack_dice="1d10", damage_modifier=0)
monster = Character(name="Monster", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr = get_dpr(player, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("WIZARD")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))